In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mlp
import glob
from tqdm import tqdm
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
from matplotlib import ticker
import geopandas as gp
import matplotlib.dates as mdates

plt.style.use('seaborn-whitegrid')
sns.set_style("white",rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['ytick.major.width'] = 1

## Data

In [ ]:
polygons = # Series of strings, names of departments in the country.

Rij_vec = # Array of matrices, each matrix being the reconstructed reproduction operator in a certain interval of time. Nodes have polygons' order.

calendar = # Dates associated to the entries of Rij_vec. The first entry of calendar corresponds to the date of the first entry of Rij_vec.

pop_vec = # Array of populations, nodes have polygons' order. 

polygons_ind = # codes of polygons, French numbering. Example: Paris is department 75.

tot_pop = np.sum(pop_vec) 

M=len(polygons)

vt_df = # DataFrame with columns 'Date', names of departments in the country, containing the equilibrium distribution for operators at that date

xt_df= # DataFrame with columns 'Date', names of departments in the country, containing spatial distribution of cases at the date (cases/total cases)

## Select time interval, compute similarity measures

In [ ]:
start=vt_df[vt_df['Date']=='2020-12-15'].index[0]
stop=vt_df[vt_df['Date']=='2021-03-30'].index[0]
vt_df_ss=vt_df.iloc[start:stop,2:]
xt_df_ss=xt_df.iloc[start:stop,2:]

dates=pd.to_datetime(vt_df.iloc[start:stop].loc[:,'Date'])
iso_dates=pd.Series([dates.iloc[i].isocalendar() for i in range(dates.shape[0])])
iso_dates_tp=pd.Series([str(iso_dates.iloc[i][0])+str(' W')+str(format(int(iso_dates.iloc[i][1]),'02d')) for i in range(dates.shape[0])])

In [ ]:
v_bar=vt_df_ss.mean()
v_bar=v_bar/v_bar.sum()
v_bar_norm=np.linalg.norm(v_bar)
angles_vv=[]
cos_vv=[]
angles_xv=[]
cos_xv=[]
for i in range(vt_df_ss.shape[0]):
    a=vt_df_ss.iloc[i]
    b=xt_df_ss.iloc[i]
    angles_vv.append(np.arccos(np.dot(a,v_bar)/(np.linalg.norm(a)*v_bar_norm))/np.pi*180)
    cos_vv.append(1-np.dot(a,v_bar)/(np.linalg.norm(a)*v_bar_norm))
    angles_xv.append(np.arccos(np.dot(b,v_bar)/(np.linalg.norm(b)*v_bar_norm))/np.pi*180)
    cos_xv.append(1-np.dot(b,v_bar)/(np.linalg.norm(b)*v_bar_norm))

In [ ]:
fig,ax=plt.subplots(figsize=(11,5))

shd=[4,12]
ax_=ax.twinx()
ax.plot(iso_dates_tp,cos_vv,color='darkorange')
ax.fill_between(iso_dates_tp[shd[0]:shd[1]],np.ones(shd[1]-shd[0])*.005,np.ones(shd[1]-shd[0])*-.0002,alpha=.3,color='silver')
ax_.plot(iso_dates_tp,cos_xv,color='forestgreen')

ax.set_ylabel(r'Cosine distance $v(t)$ and $\widebar{v}$',fontsize=15,color='darkorange')
ax_.set_ylabel(r'Cosine distance $x(t)$ and $\widebar{v}$',fontsize=15,color='forestgreen')
ax.tick_params(axis='y',labelsize=15)
ax.tick_params(axis='x',labelsize=15,labelrotation=90)
ax_.tick_params(axis='y',labelsize=15)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:01.3f}"))
ax_.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:01.2f}"))
ax.set_ylim([-.0002,.005])
ax_.set_ylim([.49,.75])
ax.set_xlim(['2020 W51','2021 W12'])
ax_.set_xlim(['2020 W51','2021 W12'])
fig.tight_layout()
#fig.savefig('C:/Users/piero/Documents/PHD/eugenio/PLOTS/Fig2/Fig2_twinx_July_w')

# Maps

In [ ]:
shp = gp.read_file('C:/Users/piero/Documents/PCS/INTERNSHIP/Letteratura/France_map/dept/dept.shp')
ss = '+proj=lcc +lat_1=49 +lat_2=44 +lat_0=46.5 +lon_0=3 +x_0=700000 +y_0=6600000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs '
shp = shp.to_crs(ss)

# Remove Corsica
shp.drop(shp.index[shp['dept'] == '2A'], inplace = True)
shp.drop(shp.index[shp['dept'] == '2B'], inplace = True)
shp=shp.reset_index(drop=True)

shp = shp.assign(centroid=shp.centroid)

shp['dept'] = shp['dept'].astype('category')
shp['dept'] = shp['dept'].cat.set_categories(polygons_ind)
shp=shp.sort_values('dept').reset_index(drop=True)

In [ ]:
start_x=xt_df[xt_df['Date']=='2021-01-12'].copy()
start_x=start_x.iloc[:,2:]
start_x.columns=polygons_ind
start_x=pd.Series(start_x.iloc[0])
start_x=list(start_x)

end_x=xt_df[xt_df['Date']=='2021-03-02'].copy()
end_x=end_x.iloc[:,2:]
end_x.columns=polygons_ind
end_x=pd.Series(end_x.iloc[0])
end_x=list(end_x)

In [ ]:
shp_start_=shp.copy()
shp_start_['start_x']=(start_x)
shp_start_['end_x']=(end_x)
shp_start_['v_bar']=(list(v_bar))
shp_start_

# Inset

In [ ]:
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

In [ ]:
fig,ax=plt.subplots(figsize=(11,5),dpi=300)

shd=[4,12]
ax_=ax.twinx()
ax.plot(iso_dates_tp,cos_vv,color='darkorange',linewidth=4)
ax.fill_between(iso_dates_tp[shd[0]:shd[1]],np.ones(shd[1]-shd[0])*.005,np.ones(shd[1]-shd[0])*-.0002,alpha=.3,color='silver')
ax_.plot(iso_dates_tp,cos_xv,color='forestgreen',linewidth=4)

ax.set_ylabel(r'Cosine distance between $v(t)$ and $\widebar{v}$',fontsize=15,color='darkorange')
ax_.set_ylabel(r'Cosine distance between $x(t)$ and $\widebar{v}$',fontsize=15,color='forestgreen')
ax.tick_params(axis='y',labelsize=15)
ax.tick_params(axis='x',labelsize=15,labelrotation=90)
ax_.tick_params(axis='y',labelsize=15)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:01.3f}"))
ax_.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:01.2f}"))
ax.set_ylim([-.0002,.005])
ax_.set_ylim([.49,.75])
ax.set_xlim(['2020 W51','2021 W12'])
ax_.set_xlim(['2020 W51','2021 W12'])

box=[0.509, 0.578, 0.4, 0.4]
box_=[0.699, 0.578, 0.4, 0.4]
axins = ax.inset_axes(box)
axins_ = ax.inset_axes(box_)
axins.get_xaxis().set_ticks([])
axins_.get_xaxis().set_ticks([])
axins.get_yaxis().set_ticks([])
axins_.get_yaxis().set_ticks([])
axins.set(facecolor='w')
axins_.set(facecolor='w')

min_value_=min(np.min(shp_start_['start_x']),np.min(shp_start_['end_x']))#,np.min(shp_start['v_bar']))
max_value_=max(np.max(shp_start_['start_x']),np.max(shp_start_['end_x']))#,np.max(shp_start['v_bar']))
cmap='YlGn'
shp_start_.plot(ax=axins, column='start_x', lw=0.2, legend=None,cmap=cmap, norm=mlp.colors.LogNorm(vmin=min_value_,vmax=max_value_))
shp_start_.plot(ax=axins_, column='end_x', lw=0.2, legend=None,cmap=cmap, norm=mlp.colors.LogNorm(vmin=min_value_,vmax=max_value_))#,legend_kwds={'label': 'Log(x)', 'shrink': 0.7})

cbar_axim = fig.add_axes([0.648, 0.52, 0.2, 0.02])
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mlp.colors.LogNorm(vmin=min_value_,vmax=max_value_))
sm.set_array([])
cbar = plt.colorbar(sm,shrink=.2,cax=cbar_axim,orientation='horizontal')#,label='$Log(x)$')
cbar.ax.tick_params(labelsize=12,direction='in')
#cbar.locator = ticker.MaxNLocator(10)  
minor_tick_locs = np.logspace(np.log10(min_value_), np.log10(max_value_), num=10)
minor_locator = ticker.LogLocator(numticks=10)# subs=minor_tick_locs)
cbar.ax.xaxis.set_major_locator(minor_locator)
cbar.set_label(r'$x$',fontsize=12)

ax.plot([0.785,0.988],[0.1,0.577],transform=ax.transAxes,color='k',linewidth=.8)
ax.plot([0.284,0.621],[0.738,0.978],transform=ax.transAxes,color='k',linewidth=.8)
#fig.tight_layout()
#fig.savefig('C:/Users/piero/Documents/PHD/eugenio/PLOTS/Fig2/Fig2_twinx_maps_line',bbox_inches='tight')
#fig.savefig('C:/Users/piero/Documents/PHD/eugenio/PLOTS/paper/Figure1a.svg',bbox_inches='tight')
#fig.savefig('C:/Users/piero/Documents/PHD/eugenio/PLOTS/paper/Figure1a.pdf',bbox_inches='tight')